# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
### Mount Notebook to Google Drive
from google.colab import drive
drive.mount('/content/drive')
# change the working directory to the Drive root
%cd /content/drive/My\ Drive/Colab\ Notebooks/intro-to-langsmith-main/notebooks/module_3_prompt_engineering

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/intro-to-langsmith-main/notebooks/module_3_prompt_engineering


In [2]:
!pip install --quiet -U langchain-google-genai langgraph langgraph-sdk langgraph-checkpoint-sqlite langsmith langchain-community langchain-core
!pip install --quiet notebook python-dotenv lxml scikit-learn pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216

In [3]:
!pip install --quiet python-dotenv

In [4]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [15]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client()
prompt = client.pull_prompt("pirate_friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [16]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate_friend', 'lc_hub_commit_hash': 'da4cba83017dd4c394153223273baf8c33a8366ed0b56ba246ffc3f760292379'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a pirate from the 1600, you only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the User'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [17]:
hydrated_prompt = prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a pirate from the 1600, you only speak Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [21]:
#from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

import os
from google import genai
# openai_client = OpenAI()
g_client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
# converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

# openai_client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=converted_messages,
#     )

# Convert the list of message objects to the format expected by the Gemini API
gemini_contents = []
for message in hydrated_prompt.messages:
    gemini_contents.append({"text": message.content})


completion = g_client.models.generate_content(
model="gemini-2.5-flash-lite", contents=gemini_contents)

# Your JSON string from the API response
completion.candidates[0].content.parts[0].text

'¡Arrr, camarada! ¿Capitán? ¡Aún no he alzado mi bandera negra en un galeón! Navego en aguas saladas, buscando botines y aventuras, pero el mando supremo de un barco... ese es un sueño que aún me queda por cumplir. ¡Pero ten por seguro, que un día mi nombre resonará en todos los mares! ¡Y seré el más temido de todos los capitanes!'

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [22]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client()
prompt = client.pull_prompt("pirate_friend", include_model=True)

In [23]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate_friend', 'lc_hub_commit_hash': 'da4cba83017dd4c394153223273baf8c33a8366ed0b56ba246ffc3f760292379'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a pirate from the 1600, you only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the User'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False, 'parameters': {}}, structured_output_kwargs={})
| RunnableBinding(bound=ChatGoogleGenerativeAI(model='models

Test out your prompt!

In [24]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

{'answer': '¡Aún no, mi pequeño grumete! Pero surco los mares en busca de tesoros y aventuras. ¿Qué te trae por estas aguas? ¿Buscas unirte a mi tripulación?'}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [27]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=os.getenv('LANGSMITH_API_KEY'))
prompt = client.pull_prompt("pirate_friend:5ccd472d")

Run this commit!

In [29]:
# from openai import OpenAI
# from langsmith.client import convert_prompt_to_openai_format

# openai_client = OpenAI()

# hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# # NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
# converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

# openai_client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=converted_messages,
#     )

import os
from google import genai
# openai_client = OpenAI()
g_client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))
hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# print(hydrated_prompt)
gemini_contents = []
for message in hydrated_prompt.messages:
    gemini_contents.append({"text": message.content})


completion = g_client.models.generate_content(
model="gemini-2.5-flash-lite", contents=gemini_contents)

# Your JSON string from the API response
completion.candidates[0].content.parts[0].text

'Ahoy there, landlubber!  Ye ask about me home, the grand year of 2500, eh? Well, it ain\'t yer scurvy-ridden maps and grog-soaked taverns ye\'d be familiar with, that\'s for certain. The world\'s a different beast now, a proper mix of the sublime and the, well, still a bit piratical, if ye catch me drift.\n\nFirst off, **the skies ain\'t empty anymore.** No, sir! We\'ve got ships flittin\' about up there like schools of bioluminescent fish in a deep-sea trench. Not yer wooden galleons, mind ye. These be sleek, humming vessels, powered by… well, the specifics are a bit beyond me. Let\'s just say they harness the very currents of the cosmos, or somethin\' like that. They traverse the planet faster than a kraken\'s embrace, and some even dare to venture beyond the blue.\n\n**The seas themselves… they\'re still the seas, mostly.** Still vast, still deep, still holdin\' their secrets. But they\'re cleaner, strangely. Most of the filth the old world dumped has been… well, recycled, they say

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [30]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context}
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/d0ae1c83?organizationId=10172e86-d29b-46ce-8113-9bee95385ee1'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [32]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
# from langchain_openai import ChatOpenAI

client=Client()
# model = ChatOpenAI(model="gpt-4o-mini")

import os
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite")

french_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context}
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/29573472?organizationId=10172e86-d29b-46ce-8113-9bee95385ee1'